Core code for preparing dataset and forming the CNN architecture

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
testset = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

Code for evaluating our CNN model

In [2]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(trainloader)
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    scheduler.step()

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"The Accuracy Achieved in this CNN model is: {test_accuracy:.2f}%")

Epoch 1, Loss: 0.1525, Accuracy: 95.14%
Epoch 2, Loss: 0.0651, Accuracy: 98.12%
Epoch 3, Loss: 0.0492, Accuracy: 98.53%
Epoch 4, Loss: 0.0432, Accuracy: 98.72%
Epoch 5, Loss: 0.0396, Accuracy: 98.90%
Epoch 6, Loss: 0.0201, Accuracy: 99.40%
Epoch 7, Loss: 0.0153, Accuracy: 99.56%
Epoch 8, Loss: 0.0133, Accuracy: 99.61%
Epoch 9, Loss: 0.0122, Accuracy: 99.63%
Epoch 10, Loss: 0.0119, Accuracy: 99.67%
The Accuracy Achieved in this CNN model is: 98.85%


Core Code for preparing the dataset for active learning model

In [3]:
from sklearn.metrics import pairwise_distances
from scipy.stats import entropy
import numpy as np

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
testset = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

Defining function for calculating uncertainty metrics

In [4]:
def calculate_uncertainty_metrics(outputs):
    outputs_tensor = torch.from_numpy(outputs)
    probabilities = F.softmax(outputs_tensor, dim=1)

    least_confidence = 1 - probabilities.max(dim=1).values.cpu().detach().numpy()

    current_probs = probabilities.clone().detach()
    current_probs[current_probs == 0] = 1e-10

    prediction_entropy = -torch.sum(current_probs * torch.log(current_probs), dim=1).cpu().detach().numpy()

    margin_sampling = 1 - (torch.max(probabilities, dim=1).values.cpu().detach().numpy() - \
                           torch.min(probabilities, dim=1).values.cpu().detach().numpy())

    return least_confidence, prediction_entropy, margin_sampling

Defining function for calculating diversity metrics

In [5]:
def calculate_diversity_metrics(features, m=5):
    feature_distances = pairwise_distances(features.cpu().detach().numpy(), metric='cosine')
    cosine_similarity = 1 - feature_distances[:, 1:m+1].mean(axis=1)

    l2_distances = pairwise_distances(features.cpu().detach().numpy(), metric='euclidean')
    l2_norm = l2_distances[:, 1:m+1].mean(axis=1)

    return cosine_similarity, l2_norm

Defining function for calculating KL divergence

In [6]:
def calculate_kl_divergence(outputs, feature_distances, m=5):
    kl_divergence = []
    for i in range(len(outputs)):
        current_sample_prob = F.softmax(outputs[i], dim=0)
        neighbor_indices = feature_distances[i, 1:m+1].astype(int)
        neighbors_prob = torch.mean(F.softmax(outputs[neighbor_indices], dim=1), dim=0)
        epsilon = 1e-10
        current_sample_prob += epsilon
        neighbors_prob += epsilon
        kl_divergence.append(F.kl_div(torch.log(current_sample_prob), neighbors_prob, reduction='batchmean'))
    return kl_divergence

Defining function for calculating metrics

In [7]:
def calculate_metrics(outputs, features, m=5):
    outputs_cpu = outputs.cpu()

    least_confidence, prediction_entropy, margin_sampling = calculate_uncertainty_metrics(outputs_cpu.detach().numpy())
    least_confidence_list.extend(torch.from_numpy(least_confidence))
    prediction_entropy_list.extend(torch.from_numpy(prediction_entropy))
    margin_sampling_list.extend(torch.from_numpy(margin_sampling))

    features_normalized = F.normalize(features, p=2, dim=1)
    cosine_similarity, l2_norm = calculate_diversity_metrics(features_normalized)

    cosine_similarity_list.extend(torch.from_numpy(cosine_similarity))
    l2_norm_list.extend(torch.from_numpy(l2_norm))

    feature_distances = pairwise_distances(features.cpu().detach().numpy(), metric='cosine')
    kl_divergence_scores = calculate_kl_divergence(outputs, feature_distances, m=5)
    kl_divergence_list.extend(kl_divergence_scores)

Code for evaluating active learning enhanced model

In [8]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(trainloader)
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    scheduler.step()

model.eval()
correct = 0
total = 0
least_confidence_list= []
prediction_entropy_list = []
margin_sampling_list = []
cosine_similarity_list = []
l2_norm_list = []
kl_divergence_list = []

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        conv1_output = model.conv1(inputs)
        relu_output = model.relu(conv1_output)
        maxpool_output = model.pool(relu_output)
        conv2_output = model.conv2(maxpool_output)
        features = model.flatten(conv2_output)
        features = features.view(features.size(0), -1)
        calculate_metrics(outputs, features)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

print(f"Average Least Confidence: {torch.mean(torch.stack(least_confidence_list))}")
print(f"Average Prediction Entropy: {torch.mean(torch.stack(prediction_entropy_list))}")
print(f"Average Margin Sampling: {torch.mean(torch.stack(margin_sampling_list))}")
print(f"Average Cosine Similarity: {torch.mean(torch.stack(cosine_similarity_list))}")
print(f"Average L2 Norm: {torch.mean(torch.stack(l2_norm_list))}")
print(f"Average KL Divergence: {torch.mean(torch.stack(kl_divergence_list))}")

Epoch 1, Loss: 0.2267, Accuracy: 93.14%
Epoch 2, Loss: 0.1040, Accuracy: 97.06%
Epoch 3, Loss: 0.0894, Accuracy: 97.51%
Epoch 4, Loss: 0.0776, Accuracy: 97.78%
Epoch 5, Loss: 0.0755, Accuracy: 97.97%
Epoch 6, Loss: 0.0524, Accuracy: 98.56%
Epoch 7, Loss: 0.0454, Accuracy: 98.73%
Epoch 8, Loss: 0.0429, Accuracy: 98.81%
Epoch 9, Loss: 0.0426, Accuracy: 98.86%
Epoch 10, Loss: 0.0409, Accuracy: 98.83%
Test Accuracy: 98.80%
Average Least Confidence: 0.007506141439080238
Average Prediction Entropy: 0.023189278319478035
Average Margin Sampling: 0.007519974373281002
Average Cosine Similarity: 0.7622093558311462
Average L2 Norm: 0.5898517370223999
Average KL Divergence: 1.379577875137329
